# RNN
```
- 데이터 -> 토큰화&정수 인코딩 -> 시퀀스 패딩
- baseline 일반신경망 Dense
- Simple RNN 기울기소실 문제
- Bidirectional LSTM: 양방향 학습
```

> - Tokenization <br>
>   - 텍스트를 숫자로 변환 <br>
> - Word Embedding <br>
>   - 단어를 고차원 벡터로 변환 <br>
> - Sequence Padding
>   - 길이가 다른 문장을 같은 크기로 고정

> - LSTM
>   - RNN의 경사소실 문제 해결
> - Bidirectional LSTM
>   - 양방향 컨텐츠 학습

```
토큰화: 숫자로 변환하는 과정
<토크나이저 3단계>
1. fit_on_text(texts) 가장 빈도가 높은 단어의 인덱스를 구축 -> 딕셔너리
2. texts_to_sequence(texts) 각문서를 정수 시퀀스로 변환
3. pad_sequence() 길이 정규화(같은 길이로)
```

딥러닝에서 Word Embedding : 각 단어를 고정된 크기의 실수 벡터로

In [1]:
sample_reviews = [
    "this movie is great and wonderful",
    "bad movie with poor acting",
    "great movie absolutely wonderful"
]

In [2]:
# 파이토치 버전
# 토큰화
from collections import Counter
import numpy as np
import torch

### 1. 단어 분할 및 빈도

In [3]:
# 1. 단어 분할 및 빈도 계산
all_words = []
for i in [review.split() for review in sample_reviews]:
    all_words.extend(i)

# 단어빈도
word_freq = Counter(all_words)
word_freq

Counter({'this': 1,
         'movie': 3,
         'is': 1,
         'great': 2,
         'and': 1,
         'wonderful': 2,
         'bad': 1,
         'with': 1,
         'poor': 1,
         'acting': 1,
         'absolutely': 1})

### 2. Tokenizer

In [4]:
# 2. Tokenizer 구현
class SimpleTokenizer:
    def __init__(self, num_words=10, oov_token = 'UNK'):
        self.num_words = num_words
        self.oov_token = oov_token
        self.word_index = {}
        self.index_word = {}
    def fit_on_texts(self, texts):
        '''문장을 단어 인덱스로 변환 '''
        all_words = []
        for i in [review.split() for review in sample_reviews]:
            all_words.extend(i)
        word_freq = Counter(all_words)
        # 빈도 높은 순서로 인덱스 부여
        # oov 토큰을 1로 설정
        self.word_index[self.oov_token] =1
        self.index_word[1]  = self.oov_token
        idx = 2
        for word, _ in word_freq.most_common(self.num_words-1):
            self.word_index[word] = idx
            self.index_word[idx] = word
            idx += 1
    def texts_to_sequences(self, texts):
        '''텍스트를 정수 시퀀스로 변환'''
        sequences = []
        for text in texts:
            seq = []
            for word in text.split():
                # 단어가 vocabulary에 있으면 인덱스를 사용 없으면 oov
                word_index = self.word_index.get(word, 1)
                seq.append(word_index)
            sequences.append(seq)
        return sequences
# Tokenizer 생성 및 학습
tokenizer =  SimpleTokenizer(num_words=10, oov_token='UNK')
tokenizer.fit_on_texts(sample_reviews)
tokenizer.word_index

{'UNK': 1,
 'movie': 2,
 'great': 3,
 'wonderful': 4,
 'this': 5,
 'is': 6,
 'and': 7,
 'bad': 8,
 'with': 9,
 'poor': 10}

In [5]:
# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(sample_reviews)
sequences

[[5, 2, 6, 3, 7, 4], [8, 2, 9, 10, 1], [3, 2, 1, 4]]

In [6]:
sample_reviews

['this movie is great and wonderful',
 'bad movie with poor acting',
 'great movie absolutely wonderful']

### 3. Padding

In [7]:
# 패딩 구현 - 문자열의 길이를 동일하게 맞춘다
def pad_sequence_manual(sequences, max_len=10, padding='pre',value=0):
    '''패딩구현'''
    padded = []
    for seq in sequences:
        if len(seq) >= max_len:
            if padded == 'pre':
                padded_seq = seq[-max_len:]
            else:
                padded_seq = seq[:max_len]
        else:
            pad_length = max_len-len(seq)
            if padding == 'pre':
                padded_seq = [value]*pad_length + seq
            else:
                padded_seq = seq + [value]*pad_length
        padded.append(padded_seq)
    return np.array(padded)
padded = pad_sequence_manual(sequences)
padded

array([[ 0,  0,  0,  0,  5,  2,  6,  3,  7,  4],
       [ 0,  0,  0,  0,  0,  8,  2,  9, 10,  1],
       [ 0,  0,  0,  0,  0,  0,  3,  2,  1,  4]])

In [8]:
# pytorch tensor 변환
sequence_tensor =  torch.LongTensor(padded)
sequence_tensor

tensor([[ 0,  0,  0,  0,  5,  2,  6,  3,  7,  4],
        [ 0,  0,  0,  0,  0,  8,  2,  9, 10,  1],
        [ 0,  0,  0,  0,  0,  0,  3,  2,  1,  4]])

### 4. 워드 임베딩

In [20]:
# 워드 임베딩 - 시각화
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

print(f'패딩된 시퀀스 형태: {sequence_tensor.shape}')
print(f'첫번째: {sequence_tensor[0]}')

패딩된 시퀀스 형태: torch.Size([3, 10])
첫번째: tensor([0, 0, 0, 0, 5, 2, 6, 3, 7, 4])


In [19]:
import torch.nn as nn

# pytorch embedding: 레이어 생성
# num_embeddings: 어휘의 크기
# embedding_dim: 각 단어를 몇 차원 벡터로 표현할 것인지
# padding_idx: 길이 맞출 때 채우는 값

embedding_layer = nn.Embedding(num_embeddings=1000, embedding_dim=8, padding_idx=0)
embedded = embedding_layer(sequence_tensor)

print(f'입력형태: {sequence_tensor.shape}')
print(f'출력형태: {embedded.shape}')

입력형태: torch.Size([3, 10])
출력형태: torch.Size([3, 10, 8])


In [17]:
# 임베딩 벡터 상세 분석
# sample data의 첫 3개 단어 임베딩

for word_idx in range(3):
  embedding_vec = embedded[0,word_idx].detach().numpy()
  word_id = sequence_tensor[0,word_idx].item()
  print(f'단어 id {word_id} : {embedding_vec}')

단어 id 0 : [0. 0. 0. 0. 0. 0. 0. 0.]
단어 id 0 : [0. 0. 0. 0. 0. 0. 0. 0.]
단어 id 0 : [0. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
# 임베딩 행렬
embedding_matrix = embedding_layer.weight.detach().numpy()

print(f'임베딩 행렬 형태: {embedding_matrix.shape}') #1000,8
print(f'패딩(id=0)의 임베딩: {embedding_matrix[0]}')
print(f'단어(id=5)의 임베딩: {embedding_matrix[5]}')

임베딩 행렬 형태: (1000, 8)
패딩(id=0)의 임베딩: [0. 0. 0. 0. 0. 0. 0. 0.]
단어(id=5)의 임베딩: [ 0.62010115  0.93770546 -2.8720152   2.1677155  -0.77353656  0.5262479
 -2.0368896  -1.463761  ]


---

## RNN 적용

In [ ]:
class RnnModule(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RnnModule, self).__init__()
        self.embdding = nn.Embedding(vocab_size,embedding_dim,padding_idx=0)
        self.rnn  = nn.RNN(embedding_dim,hidden_dim,batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x_emb =  self.embdding(x)  #(batch,seq_len, embedding_dim)
        rnn_out, h_n = self.rnn(x_emb) # rnn_out(batch,seq_len,hidden_dim)
                                     # h_n  (1, batch,hidden_dim)
        # 마지막 스텝의 출력
        last_output = rnn_out[:,-1,:]   # (batch,hidden_dim)
        output = self.sigmoid(self.fc(last_output))
        return output